In [ ]:
# importing libraries

In [ ]:
import torch
import intel_extension_for_pytorch as ipex
print(torch.backends.mkl.is_available())

In [ ]:
from distributed import Client
client = Client()

In [ ]:
import modin.pandas as pd

In [ ]:
df = pd.read_excel("IPC_Dataset.xlsx")

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
data_dict = df.to_dict('list')  # convert DataFrame to dictionary of lists
dataset = Dataset.from_dict(data_dict)

In [ ]:
# data processing

In [ ]:
ds = dataset.train_test_split(test_size=0.01)

In [ ]:
from transformers import AutoTokenizer

checkpoint = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def format_instruction(chapter: str, section: str):
	return f"""### Instruction:
What does {section.strip()}  about?
"""


In [ ]:
def generate_instruction_dataset(data_point):

    return {
        "chapter": data_point["Chapter"],
        "Section": data_point["Section"],
        # "Title": data_point["Title"],
        # "Description": data_point["Description"],
        "text": format_instruction(data_point["Chapter"],data_point["Section"])
    }

In [ ]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset)
    )

In [ ]:
ds["train"] = process_dataset(ds["train"])

In [ ]:
ds["train"]["text"][0]

In [ ]:
def format_instruction1(title: str, Description : str):
	return f"""
# {title.strip()} which describes {Description.strip()}
# """.strip()

In [ ]:
def generate_instruction_dataset1(data_point):

    return {
        "chapter": data_point["Chapter"],
        "Section": data_point["Section"],
        "Title": data_point["Title"],
        "Description": data_point["Description"],
        "text": format_instruction1(data_point["Title"],data_point["Description"])
    }

In [ ]:
def process_dataset1(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset1)
    )

In [ ]:
ds["out"] = process_dataset1(ds["train"])

In [ ]:
ds["out"]["text"][0]

In [ ]:
ds["title"] = ds["train"]["text"]

In [ ]:
ds["res"] = ds["out"]["text"]

In [ ]:
dfs= pd.DataFrame()

In [ ]:
dfs["title"] = ds["title"] 

In [ ]:
dfs["response"] = ds["res"]

In [ ]:
dfs["title"][3]

In [ ]:
dfs["response"][3]

In [ ]:
fin = Dataset.from_pandas(dfs)

In [ ]:
finalDs= fin.train_test_split(test_size=0.01)

In [ ]:

def preprocess_function(examples):
    inputs = [doc for doc in examples["title"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["response"], max_length=500, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_ds = finalDs.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# finetuning LLM

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="finalIPC",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=5,
    predict_with_generate=True,
    push_to_hub=True
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)

In [ ]:
trainer.train()